## <span style='color:#ff5f27'> 📝 Imports

In [1]:
from xgboost import XGBClassifier

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [2]:
import hopsworks
project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/148
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [3]:
# Retrieve the 'transactions_view_fraud_batch_fv' feature view
feature_view = fs.get_feature_view(
    name='transactions_view_fraud_batch_fv',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [4]:
# Get the model registry
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


## <span style='color:#ff5f27'>🚀 Fetch and test the model</span>

Finally you can start making predictions with your model! Retrieve your model from Hopsworks model registry.

In [5]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="xgboost_fraud_batch_model",
    version=1,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

In [6]:
# Initialize the model
model = XGBClassifier()

# Load the model from a saved JSON file
model.load_model(saved_model_dir + "/model.json")
model

XGBClassifier(base_score='1.1981872E-1', booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['int', 'float', 'int', 'int', 'float', 'float',
                             'float', 'float', 'float', 'int'],
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

---
## <span style="color:#ff5f27;">🔮  Batch Prediction and Logging </span>


In [17]:
# Initialize batch scoring
feature_view.init_batch_scoring(1)

# Get batch data for logging
batch_data = feature_view.get_batch_data()

# Display the first 3 rows
batch_data.head(3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.70s) 


,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg,datetime
0,0,94.39,79,1125,0.000000,0.0,94.39,1.0,0.000000,1641010897000
1,1,72.32,79,1125,0.154888,0.0,72.32,1.0,0.154888,1641021603000
2,2,84.45,79,1123,0.223899,0.0,84.45,1.0,0.223899,1641194862000


In [18]:
# Use the retrieved XGBoost model to make predictions on the batch data
predictions = model.predict(batch_data)

# Display the first five predictions
predictions[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# stop the job materialization schedule and materialize log manually
feature_view.pause_logging()
feature_view.materialize_log(wait=True)

In [ ]:
# read untransformed log
feature_view.read_log().head(3)

In [ ]:
# read transformed log
feature_view.read_log(transformed=True).head(3)